# TV Script Generation

In this project, I have generated [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using Long Short Term Memory (LSTM) network. I have used a part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network I have built generates a new ,"fake" TV script, based on patterns it recognized in this training data.

The data is in `./data/Seinfeld_Scripts.txt`. 

In [1]:
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
We can set `view_line_range` below to different range to view different parts of the data. 

In [2]:
view_line_range = (100, 110)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 100 to 110:
george: ...and, and maybe shell see me when she gets there, does this woman intend to spend time with me? 

claire: id have to say, uh, no. 

(george shows his note-block to jerry; it says very largely: no.) 

claire: to be polite. 

george: to be polite. i rest my case. 



## Pre-processing Functions

### Lookup Table
To create a word embedding, we first need to transform the words to ids.  In this function, I created two dictionaries:
- Dictionary to go from the words to an id (`vocab_to_int`)
- Dictionary to go from the id to word (`int_to_vocab`)

In [3]:
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    sorted_text = sorted(list(set(text)))
    vocab_to_int = {j:i for i, j in enumerate(sorted_text)}
    int_to_vocab = {value:key for key, value in vocab_to_int.items()}
    # return tuple of two dictionaries
    return (vocab_to_int, int_to_vocab)

tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
Punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

I have implemented the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word.

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    token_lookup = {'.': '||period||',
                   ',': '||comma||',
                   '"': '||quotation_mark||',
                   ';': '||semi_colon||',
                   '!': '||exclamation_mark||',
                   '?': '||question_mark||',
                   '(': '||left_parenthesis||',
                   ')': '||right_parenthesis||',
                   '-': '||dash||',
                   '\n': '||return||'}    
    return token_lookup

tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

In [5]:
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point 1
We can stop running the notebook after executing the above cell and resume it from below cell

In [6]:
# Load the saved data
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [7]:
print(int_text[:5])
print({k:vocab_to_int[k] for k in list(vocab_to_int.keys())[100:105]})
print({k:int_to_vocab[k] for k in list(int_to_vocab.keys())[100:105]})
print({k:token_dict[k] for k in list(token_dict.keys())[:5]})

[19081, 10247, 13592, 21378, 21378]
{"'costanza'": 100, "'cotton": 101, "'course": 102, "'crook": 103, "'cuz": 104}
{100: "'costanza'", 101: "'cotton", 102: "'course", 103: "'crook", 104: "'cuz"}
{'.': '||period||', ',': '||comma||', '"': '||quotation_mark||', ';': '||semi_colon||', '!': '||exclamation_mark||'}


## Build the Neural Network
In this section, I have built the components necessary to build an LSTM by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [8]:
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
I have started with the preprocessed input data. I have used [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to my dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.


### Batching
I have implemented the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Our first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [9]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    X_tensors, y_tensors = [], []
    for i in range(len(words) - sequence_length):
        try:
            end_index = i + sequence_length
            feature = words[i:end_index]
            # print(feature)
            X_tensors.append(feature)
            target = words[end_index]
            # print(target)
            y_tensors.append(target)
        except:
            break
    feature_target_data = TensorDataset(torch.from_numpy(np.asarray(X_tensors)), 
                         torch.from_numpy(np.asarray(y_tensors)))
    # create dataloader
    data_loader = DataLoader(feature_target_data, shuffle=False, batch_size=batch_size)
    return data_loader 

### Test dataloader 

In [10]:
test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()
print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[  0,   1,   2,   3,   4],
        [  1,   2,   3,   4,   5],
        [  2,   3,   4,   5,   6],
        [  3,   4,   5,   6,   7],
        [  4,   5,   6,   7,   8],
        [  5,   6,   7,   8,   9],
        [  6,   7,   8,   9,  10],
        [  7,   8,   9,  10,  11],
        [  8,   9,  10,  11,  12],
        [  9,  10,  11,  12,  13]])

torch.Size([10])
tensor([  5,   6,   7,   8,   9,  10,  11,  12,  13,  14])


## Build the Neural Network
I have implemented an LSTM using PyTorch's [Module nn](http://pytorch.org/docs/master/nn.html#torch.nn.Module). I have implemented the following classes.
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM hidden state
 - `forward` - Forward propagation function.

**The output of this model is the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

In [11]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocabulary_size, output_size, embedding_dimension, 
                 hidden_dimension, n_layers, drop_out=0.5):
        """
        Initialize the RNN Module from pytorch
        :param vocabulary_size: dimensions of input to the neural network         
        :param output_size: dimensions of output from the neural network
        :param embedding_dimension: The size of embeddings if used        
        :param hidden_dimension: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM layers
        """
        super(RNN, self).__init__()
        
        # Output size
        self.output_size = output_size
        # Number of layers
        self.n_layers = n_layers
        # Number of hidden nodes/lstm cells
        self.hidden_dimension = hidden_dimension
        
        # model layers
        self.embedding = nn.Embedding(vocabulary_size, embedding_dimension)
        self.lstm = nn.LSTM(embedding_dimension, hidden_dimension, n_layers, dropout=drop_out, batch_first=True)
        self.fc = nn.Linear(hidden_dimension, output_size)
    
    
    def forward(self, nnet_input, hidden):
        """
        Forward propagation 
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: the output and the latest hidden state as tensors
        """
        batch_size = nnet_input.size(0)
        
        embeds = self.embedding(nnet_input)
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dimension)
        
        # fully connected layer
        output_layer = self.fc(lstm_out)
        
        # reshape out_put layer
        output_layer = output_layer.view(batch_size, -1, self.output_size)
        
        # final batch
        output = output_layer[:, -1]

        # return one batch of output and the hidden state
        return output, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dimension)
        '''
    
        # initialize hidden state with zero weights, and move to GPU if available
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden_state = (weight.new(self.n_layers, batch_size, self.hidden_dimension).
                            zero_().cuda(),
                            weight.new(self.n_layers, batch_size, self.hidden_dimension).
                            zero_().cuda())
        else:
            hidden_state = (weight.new(self.n_layers, batch_size, self.hidden_dimension).zero_(),
                            weight.new(self.n_layers, batch_size, self.hidden_dimension).zero_())        
        return hidden_state

tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Used the RNN class to apply forward and back propagation. This function will be called iteratively.

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. 

In [12]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation 
    :param rnn: pytorch module
    :param optimizer: optimizer for the neural network
    :param criterion: loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    # move data to GPU, if available
    if train_on_gpu:
        inp, target = inp.cuda(), target.cuda()
        
    hidden = tuple([each.data for each in hidden])
    
    rnn.zero_grad()
    
    output, latest_hidden = rnn(inp, hidden)
        
    ## perform backpropagation and optimization    
    
    loss = criterion(output, target)
    loss.backward()   
    nn.utils.clip_grad_norm_(rnn.parameters(), 5)
    
    optimizer.step()
    
    # return the loss over a batch and the hidden state produced by the model
    return loss.item(), latest_hidden

tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

In [13]:
def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # iterate over completely full batches only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                batches = len(train_loader)
                percent_trained = ((batch_i + (batches * (epoch_i - 1)))/(batches * n_epochs)) * 100
                print('Epoch: {:>4}/{:<4}  Loss: {:.2f}  percent_trained: {:.2f} \n'.format(
                    epoch_i, n_epochs, np.average(batch_losses), percent_trained))
                batch_losses = []
    print('Final batch loss: {:.2f}'.format(np.average(batch_losses)))

    # returns a trained rnn
    return rnn

### Hyperparameters

In [14]:
# Data params
# Sequence Length
sequence_length = 8  # of words in a sequence
# Batch Size
batch_size = 256

# data loader
train_loader = batch_data(int_text, sequence_length, batch_size)

In [15]:
# Training parameters
# Number of Epochs
epochs = 15
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dimension = 300
# Hidden Dimension
hidden_dimension = 128
# Number of RNN Layers
n_layers = 2
# Show stats for every n number of batches
show_every_n_batches = 3000

### Train

In [16]:
import time
start = time.time()
# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dimension, hidden_dimension, 
          n_layers, drop_out=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, epochs, 
                        show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

print("It took {:.2f} minutes to train".format((time.time() - start)/60))


Training for 15 epoch(s)...
Epoch:    1/15    Loss: 4.82  percent_trained: 5.74 

Epoch:    2/15    Loss: 4.22  percent_trained: 12.41 

Epoch:    3/15    Loss: 4.01  percent_trained: 19.07 

Epoch:    4/15    Loss: 3.88  percent_trained: 25.74 

Epoch:    5/15    Loss: 3.78  percent_trained: 32.41 

Epoch:    6/15    Loss: 3.71  percent_trained: 39.07 

Epoch:    7/15    Loss: 3.65  percent_trained: 45.74 

Epoch:    8/15    Loss: 3.60  percent_trained: 52.41 

Epoch:    9/15    Loss: 3.56  percent_trained: 59.07 

Epoch:   10/15    Loss: 3.53  percent_trained: 65.74 

Epoch:   11/15    Loss: 3.50  percent_trained: 72.41 

Epoch:   12/15    Loss: 3.47  percent_trained: 79.07 

Epoch:   13/15    Loss: 3.45  percent_trained: 85.74 

Epoch:   14/15    Loss: 3.42  percent_trained: 92.41 

Epoch:   15/15    Loss: 3.40  percent_trained: 99.07 

Final batch loss: 3.44
Model Trained and Saved
It took 38.61 minutes to train


/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


### hyperparameter tuning

**Answer:**
I started with the following hyper parameters. This helped me to quickly train to see if everything works fine.
- sequence_length = 8
- batch_size = 128
- epochs = 1
- learning_rate = 1
- embedding_dimension = 400 
- hidden_dimension = 200
- n_layers = 1 

This produced an validation loss of `356`.

Then I tried several hyper parameter combination before the one I chosen above which yielded the loss of `~3.5`.

I considered the following facts while choosing hyperparameters.

**Number of epochs**: The more epochs you use, the better the model fit will be. However, having too many epochs can waste computational resources and can lead to overfitting.

**Learning rate**: If learning rate is too big, the weights tend to explode and the network fails to fit the data. A good choice to start at is 0.1. Generally, if the network has problems fitting the data, we try reducing the learning rate. In contrast, lower the learning rate, the smaller the weight updates and the longer it takes for the neural network to converge.

**hidden_dimension:** The more hidden nodes you have, the more accurate the predictions of the model will be. However, it will be harder to optimize the weights of the model, and the model is unnecessarily complex leading to overfitting. 

Apart from the above considerations, my hyper-parameter setting also involves a lot of trail and error. The dataframe below shows some of the hyper-parameters combinations I tested.

In [17]:
import pandas as pd
hyper_parameters = dict()
row_index = ['sequence_length', 'batch_size', 'epochs','learning_rate', 
             'embedding_dimension', 'hidden_dimension','n_layers', 'loss']
hyper_parameters['model_1'] =[str(i) for i in [8, 256, 6, 0.01, 300, 128, 2, 4.14]]
hyper_parameters['model_2'] = [str(i) for i in [8, 256, 10, 0.03, 300, 128, 2, 3.62]]
hyper_parameters['model_3'] = [str(i) for i in [8, 256, 10, 0.0015, 300, 128, 2, 3.56]]
hyper_parameters['model_4'] = [str(i) for i in [8, 256, 15, 0.001, 300, 128, 2, 3.44]]
pd.DataFrame(hyper_parameters, index=row_index)

,model_1,model_2,model_3,model_4
sequence_length,8,8,8,8
batch_size,256,256,256,256
epochs,6,10,10,15
learning_rate,0.01,0.03,0.0015,0.001
embedding_dimension,300,300,300,300
hidden_dimension,128,128,128,128
n_layers,2,2,2,2
loss,4.14,3.62,3.56,3.44


---
# Checkpoint 2

In [18]:
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. `generate` function takes a word id to start with (`prime_id`), and generates a set length of text (`predict_len`). It uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [19]:
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. We need to set `gen_length` to the length of TV script we want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

We can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (We can also start with any other names we find in the original text file!).

In [20]:
# run the cell multiple times to get different results!
gen_length = 500 # modify the length to your preference
prime_word = 'kramer' # name for starting the script


pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


kramer:) yeah!

kramer: yeah, yeah, that's it! it's a good time for a friend of bystanding.

kramer: yeah, yeah-

jerry: what?

jerry:(to elaine) what?

jerry: oh.

jerry: oh, no no no no, no. it's just to be the same thing.

elaine:(to jerry) i can't believe i can do that, i'm geraldo...

jerry: i don't know.

jerry: oh, i'm sorry.

jerry:(pointing to the waitress) hey, you know, this was the most important thing i have behaved to the supermarket.

george: what?

elaine: well, i can't... i don't have to talk to you.

jerry:(to the phone) what are you going to do?

jerry: oh, i got it in the middle of alabama, or what is that?

hoyt: no, no, no.

jerry: well, i think you were going to be on the phone.

george: i think i can go with this woman. i mean.

george:(to jerry) i think i could do it, it's like a real incriminating.

jerry: i don't know. i'm gonna go to the bathroom.

jerry: i don't want to get the bathroom.(he turns to newmans door)

george: oh yeah!

george:(to himself) you c

#### Save scripts

In [21]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()